In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install ultralytics -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.2 MB/s eta 0:00:00


In [ ]:
import gradio as gr
from ultralytics import YOLO
import cv2
import numpy as np

# Load model
model = YOLO("/content/drive/MyDrive/Yolov11 Result/cavity_yolov11m_finetuned/weights/best.pt")

print("Model Classes:", model.names)  # Should show: {0: 'Cavity', 1: 'Prosthesis', 2: 'Root Canal', 3: 'Restoration'}

def detect_cavities(image):
    img = np.array(image)
    results = model(img)[0]
    boxes = results.boxes

    annotated = img.copy()
    detections = []

    for box in boxes:
        cls = int(box.cls[0])
        conf = float(box.conf[0])
        label = model.names[cls]

        # ⭐ Detect ONLY the cavity class → class 0
        if cls == 0:
            x1, y1, x2, y2 = box.xyxy[0].tolist()

            # Draw red bounding box around cavity
            cv2.rectangle(
                annotated,
                (int(x1), int(y1)),
                (int(x2), int(y2)),
                (0, 0, 255),
                3
            )

            # Add label text
            cv2.putText(
                annotated,
                f"{label} {conf:.2f}",
                (int(x1), int(y1) - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.7,
                (0, 0, 255),
                2
            )

            detections.append(f"{label} - {conf:.2f}")

        # Ignore other classes (Prosthesis, Root Canal, Restoration)

    summary = "\n".join(detections) if detections else "No cavities detected"

    return annotated, summary


# Gradio UI
interface = gr.Interface(
    fn=detect_cavities,
    inputs=gr.Image(type="pil", label="Upload Dental X-ray"),
    outputs=[
        gr.Image(type="numpy", label="Detection Result"),
        gr.Textbox(label="Prediction Summary")
    ],
    title="🦷 Cavity Detection System (YOLOv11)",
    description="Only cavity detections (class 0) are highlighted.",
    allow_flagging="never"
)

interface.launch(debug=True)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Model Classes: {0: 'Cavity', 1: 'Prosthesis', 2: 'Root Canal', 3: 'Restoration'}


/usr/local/lib/python3.12/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://79867d42d440bd0ce5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



0: 928x1024 2 Cavitys, 5550.2ms
Speed: 47.5ms preprocess, 5550.2ms inference, 37.6ms postprocess per image at shape (1, 3, 928, 1024)
